In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm 

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
train = '/content/drive/MyDrive/Dataset/fashion-mnist_train.csv'
test = '/content/drive/MyDrive/Dataset/fashion-mnist_test.csv'

df_train = pd.read_csv(train)
df_test = pd.read_csv(test)

y_train_df = df_train[['label']]
y_test_df = df_test[['label']]

X_train_df = df_train.drop(['label'], inplace = False, axis = 1)
X_test_df = df_test.drop(['label'], inplace = False, axis = 1)

X_train = X_train_df.copy()
X_train = X_train.values

X_test = X_test_df.copy()
X_test = X_test.values

y_train = y_train_df.copy()
y_train = y_train.values

y_test = y_test_df.copy()
y_test = y_test.values

X_train = X_train/255
X_test = X_test/255

In [9]:
yint = np.zeros((60000, 10))
for i in range(len(y_train)):
    a = y_train[i]
    yint[i][a] = 1    

y_train = yint

In [10]:
def layers():
    n_X = 784
    n_h = 32
    n_y = 10
    
    return (n_X, n_h, n_y)

(n_X, n_h, n_y) = layers(X_train, y_train)

In [50]:
def params_init(n_X, n_h, n_y):
    W1 = np.random.randn(n_h, n_X) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    params = {'W1': W1, 'W2': W2, 'b1': b1, 'b2': b2}
    return params

In [51]:
def softmax(x):
    exps = np.exp(x - x.max())
    return exps / np.sum(exps, axis=0)

def forwardprop(parameters, X):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    Z1 = np.dot(W1, X.T) + b1
    A1 = np.tanh(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = (softmax(Z2))
    A2 = A2.T
        
    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}

    return A2, cache

In [52]:
def backprop(X, y, cache, parameters):
    
    Z1 = cache['Z1']
    A1 = cache['A1']
    Z2 = cache['Z2']
    A2 = cache['A2']
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    m = 60000
    dZ2 = A2 - y  
    dW2 = (1/m)*np.dot(A1, dZ2)
    db2 = (1/m)*np.sum(dZ2, axis=0, keepdims= True)
    a = np.dot(dZ2, W2)
    dZ1 = ((1-np.power(A1, 2)))*(a.T)
    dW1 = (1/m)*np.dot(dZ1, X)
    db1 = (1/m)*np.sum(dZ1, axis=1, keepdims= True)
    
    derivatives = {'dW2': dW2, 'db2': db2, 'dW1': dW1, 'db1': db1}
    
    return derivatives

In [53]:
def updates(derivatives, parameters, alpha = 1.2):
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    dW1 = derivatives['dW1']
    db1 = derivatives['db1']
    dW2 = derivatives['dW2']
    db2 = derivatives['db2']
    
    
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2.T
    b2 -= alpha * db2.T
    
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    
    return parameters

In [59]:
def fit(X, Y, n_h = 32, epochs = 10):
    
    n_x = 784
    n_y = 10
    
    parameters = params_init(n_x, n_h, n_y)
    

    for i in range(epochs):
         
        A2, cache = forwardprop(parameters, X)
 
        grads = backprop(X, Y, cache, parameters)
 
        parameters = updates(grads, parameters, alpha = 2)
        
        
        if i%100 == 0:
            print(i)

    return parameters

In [61]:
parameters = fit(X_train, y_train, 32, epochs=10000)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [62]:
a2, n = forwardprop(parameters, X_test)
score = np.zeros((10000))
for i in range(len(a2)):
    for j in range(10):
        if a2[i, j] == max(a2[i]):
            score[i] = j
sum = 0
for i in range(len(y_test)):
    if score[i] == y_test[i]:
        sum += 1
        
print(sum/100)

84.67
